Libraries required

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Reshape, Concatenate, Activation, MaxPooling2D, Flatten, Dense, Dropout, Lambda
from keras.applications.resnet import ResNet50
import tensorflow as tf
import numpy as np
from keras.utils import to_categorical

Fully connected network used to classify the final output vector after the feature maps output by the pre-trained models are concatenated

In [ ]:
def fc():
  model = Sequential([
      Flatten(),
      Dense(1000, kernel_initializer=tf.keras.initializers.GlorotNormal(seed=0)),
      Activation('relu'),
      Dropout(0.2),
      Dense(7, kernel_initializer=tf.keras.initializers.GlorotNormal(seed=0)),
      Activation('softmax'),
  ])
  return model

Connection to Google Drive to access the 224x224 variant of the dataset

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Loading the two inputs of the Siamese CNN, the two validation inputs, and the training and validation labels.

In [ ]:
train_a = np.load('/content/gdrive/MyDrive/224X224/x_Train_A.npy')
train_b = np.load('/content/gdrive/MyDrive/224X224/x_Train_B.npy')
labels = np.load('/content/gdrive/MyDrive/224X224/y_Train.npy')
val_a = np.load('/content/gdrive/MyDrive/224X224/x_Test_A.npy')
val_b = np.load('/content/gdrive/MyDrive/224X224/x_Test_B.npy')
val_labels = np.load('/content/gdrive/MyDrive/224X224/y_Test.npy')

Converting training and validation labels into categories.

In [ ]:
labels = to_categorical(labels)
val_labels = to_categorical(val_labels)

print(labels.shape)
print(val_labels.shape)

Definition of the Siamese CNN model that implements the ResNet50 pre-trained model


In [ ]:
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
resnet.trainable = True

left_input = Input(shape=(224,224,3))
right_input = Input(shape=(224,224,3))

left_features = resnet(left_input)
right_features = resnet(right_input)
print(left_features.shape)

merged_features = tf.concat([left_features, right_features], axis=-1)
print(merged_features.shape)

fc_net = fc()
output = fc_net(merged_features)
print(output.shape)

model = Model(inputs=[left_input, right_input], outputs=output)

Training of the model and definition of checkpoint method to save the best results obtained by the model.

In [ ]:
from keras.callbacks import ModelCheckpoint

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

filepath = "/content/gdrive/My Drive/best_weights_resnet50.hdf5"
checkpoint = ModelCheckpoint(filepath, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             monitor='val_loss', 
                             mode='min', 
                             verbose=1)

history = model.fit([train_a, train_b],
           labels,
           epochs=50,
           batch_size=32,
           shuffle=True,
           validation_data=([val_a, val_b], val_labels),
           callbacks=[checkpoint])

Plotting the results of accuracy and loss after the model has finished training

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

Observng the highest accuracy and lowest loss values of the model

In [ ]:
best_accuracy = max(history.history['val_accuracy'])

# Find the index of the highest accuracy value
index = history.history['val_accuracy'].index(best_accuracy)

# Get the corresponding loss value
loss_f_best_acc = history.history['val_loss'][index]

print('Best Accuracy:', best_accuracy, 'Loss:', loss_f_best_acc)

best_loss = min(history.history['val_loss'])

# Find the index of the minimum loss value
index = history.history['val_loss'].index(best_loss)

# Get the corresponding loss value
acc_f_min_loss = history.history['val_accuracy'][index]

print('Best Loss:', best_loss, 'Accuracy:', acc_f_min_loss)